In [1]:
import tkinter as tk
from tkinter import simpledialog
from tkinter import filedialog
from tkinter import messagebox
from tkinter.filedialog import askopenfilename
import pandas as pd
import numpy as np
import sys


def check_number():
    text1 = entry1.get()
    text2 = entry2.get()
    try:
        float(text1)
        float(text2)
    except ValueError:
        messagebox.showerror("Error", "Please enter a whole number")
    else:
  
        target_depth = int(entry1.get())
        num_meas_points = int(entry2.get())
        
        
        df = pd.read_excel(file_name[0], sheet_name=0)
        
        # Convert dataframe entries from strings to floats/NAs and then drop NAs
        df = df.apply(lambda x: pd.to_numeric(x, errors='coerce'))
        df.dropna(how='all', axis=0, inplace=True)

        # Reset the index
        df = df.reset_index(drop=True)

        # Set x and y
        x = 0  # every 7 rows (i.e. Min/Max/Avg Data)
        y = num_meas_points  # every N points

        # Iterate over the DataFrame in steps of y to remove average data points
        for i in range(y+1, len(df), y):
            # Delete every x rows
            df.drop(df.index[i-1:i-1+x], inplace=True) 
    
#         # Drop last averagingrows
#         df.drop(df.columns[[2,3,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49]], axis=1, inplace=True)
    
        # Only select needed data
        columns_to_keep = ["Thickness # 1 (nm)", "X (cm)", "Y (cm)"]
        df = df[columns_to_keep]
    
        # Reset index for final time
        df = df.reset_index(drop=True)

        # Set before etch depth
        df_before = df
        
        
        
        # # The same, but for second file
    
        df = pd.read_excel(file_name[1], sheet_name=0)

        # Convert dataframe entries from strings to floats/NAs and then drop NAs
        df = df.apply(lambda x: pd.to_numeric(x, errors='coerce'))
        df.dropna(how='all', axis=0, inplace=True)

        # Reset the index
        df = df.reset_index(drop=True)

        # Set x and y
        x = 0  # every 7 rows (i.e. Min/Max/Avg Data)
        y = num_meas_points  # every N points

        # Iterate over the DataFrame in steps of y to remove average data points
        for i in range(y+1, len(df), y):
            # Delete every x rows
            df.drop(df.index[i-1:i-1+x], inplace=True) 
    
#         # Drop last averagingrows
#         df.drop(df.columns[[2,3,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49]], axis=1, inplace=True)
    
        # Only select needed data
        columns_to_keep = ["Thickness # 1 (nm)", "X (cm)", "Y (cm)"]
        df = df[columns_to_keep]
    
        # Reset index for final time
        df = df.reset_index(drop=True)

        # Set after etch depth
        df_after = df

        # Subtract off After from Before
        diff_df = df[['X (cm)','Y (cm)']]
        diff_df['Etch Depth (nm)'] = df_before[['Thickness # 1 (nm)']] - df_after[['Thickness # 1 (nm)']]

        # Rounding
        diff_df=diff_df.round(2)
        

        import plotly_express as px
        import plotly.graph_objects as go
        import numpy as np
        import webbrowser


        # 3D Heatmap

        x = diff_df["X (cm)"].values
        y = diff_df["Y (cm)"].values
        z = -diff_df["Etch Depth (nm)"].values

        # Create the 3D Heatmap
        fig = px.scatter_3d(
                            x=x[0:49], 
                            y=y[0:49], 
                            z=z[0:49], 
                            color=z[0:49], 
#                             [0:50] Wafer 1
#                             [50:99] Wafer 2
#                             [100:149] Wafer 3
#                             [150:199] Wafer 4
#                             [200:249] Wafer 5
#                             [250:299] Wafer 6
#                             [300:349] Wafer 7
#                             [350:399] Wafer 8
#                             [400:449] Wafer 9
#                             [450:499] Wafer 10
                            
                            color_continuous_scale=[[0, 'red'], [0.5, 'green'], [1, 'red']]
                            )

        # Generate x, y, z coordinates for the disk
        x_disk = x
        y_disk = y
        z_disk = np.ones(len(z))*target_depth*(-1)

        # Add the disk as a new trace to the existing scatter plot
        fig.add_trace(go.Scatter3d(x=x_disk, 
                                   y=y_disk, 
                                   z=z_disk,mode='markers', 
                                   marker=dict(size=10, 
                                               color='blue', 
                                               opacity=0.10),
                                   showlegend = False
                                  )
                     )

        fig.update_layout(scene = dict(
                            xaxis_title=diff_df.columns[0],
                            yaxis_title=diff_df.columns[1],
                            zaxis_title=diff_df.columns[2]),
                            margin=dict(r=20, b=10, l=10, t=10),
                            coloraxis_colorbar_title_text = diff_df.columns[2]    
                         )

        # Open Graph in Web Browser
        fig.write_html("my_plot.html")
        webbrowser.open("my_plot.html")


           
            
# DATA INPUT, DO NOT EDIT BELOW THIS LINE EXCEPT FOR INPUT MOD        
        
def open_file():  
    add_file = filedialog.askopenfilename()
    file_name.append(add_file) 
               
def run_app():
    global entry1, entry2, canvas, file1, file2, file_name
    root = tk.Tk()
    root.title("3D Heatmap Generator")
    
    file_name = []

    # Add widgets to the window
     
    label1 = tk.Label(root, text="Enter target etch depth (nm):")
    label1.grid(row=1, column=0)
    entry1 = tk.Entry(root)
    entry1.grid(row=2, column=0)
    label2 = tk.Label(root, text="Enter the number of measurement points:")
    label2.grid(row=3, column=0)
    entry2 = tk.Entry(root)
    entry2.grid(row=4, column=0)
    open_file1_button = tk.Button(root, text="Open Initial Measurement File (.xlsx only)", command=open_file)
    open_file1_button.grid(row=5, column=0)

    open_file2_button = tk.Button(root, text="Open Final Measurement File (.xlsx only)", command=open_file)
    open_file2_button.grid(row=6, column=0)

    button = tk.Button(root, text="Run Program", command=check_number)
    button.grid(row=7, column=0)
        
    w = 230
    h = 160
    ws = root.winfo_screenwidth()
    hs = root.winfo_screenheight()
    x = (ws/2) - (w/2)
    y = (hs/2) - (h/2)
    root.geometry('%dx%d+%d+%d' % (w, h, x, y))
    root.mainloop()
run_app()